## DSC650-T301 Big Data (2235-1)
## 5/16/2023
## Joshua Greenert

## 10.1.a

In [1]:
# Create a tokenize function that splits a sentence into words. Ensure that your tokenizer removes basic punctuation.
import string

testString = "the ! fat cat sat on the stoop, and then went up."

def tokenize(sentence):
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))
    tokens = sentence.split()
    
    return tokens

In [2]:
tokenize(testString)

['the', 'fat', 'cat', 'sat', 'on', 'the', 'stoop', 'and', 'then', 'went', 'up']

## 10.1.b

In [3]:
# Implement an `ngram` function that splits tokens into N-grams. 
def ngram(tokens, n):
    ngrams = []
    # Create n-grams using a sliding window approach
    for i in range(len(tokens) - n + 1):
        ngram = tokens[i:i+n]
        ngrams.append(ngram)
    return ngrams

## 10.1.c

In [4]:
# Implement an one_hot_encode function to create a vector from a numerical vector from a list of tokens.
def one_hot_encode(tokens, num_words):
    token_index = {}
    results = []
    
    # Assign unique index to each unique token
    for token in tokens:
        if token not in token_index:
            token_index[token] = len(token_index) + 1
    
    # Create the one-hot encoded vector
    for token in tokens:
        encoding = [0] * num_words
        token_idx = token_index[token]
        encoding[token_idx - 1] = 1
        results.append(encoding)
    
    return results

## 10.2

In [ ]:
# Using listings 6.16, 6.17, and 6.18 in Deep Learning with Python as a guide, 
# train a sequential model with embeddings on the IMDB data found in data/external/imdb/. 
# Produce the model performance metrics and training and validation accuracy curves within the Jupyter notebook


## 10.3

In [ ]:
# Using listing 6.27 in Deep Learning with Python as a guide, fit the same data with an LSTM layer. 
# Produce the model performance metrics and training and validation accuracy curves within the Jupyter notebook.


## 10.4

In [ ]:
# Using listing 6.46 in Deep Learning with Python as a guide, fit the same data with a simple 1D convnet. 
# Produce the model performance metrics and training and validation accuracy curves within the Jupyter notebook.
